In [ ]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import Word2Vec
from time import time

In [ ]:
# Load the data sets
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bytes-to-text.csv', names=['raw-bytes', 'tag'], header=None)
print('Data Loaded')

Data Loaded


In [1]:
# remove those rows which are larger than 50000 word size
df['length'] = df['raw-bytes'].str.len()
lengths = df['length']
todel = lengths[lengths>50000]
df = df.drop(todel.index)
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df

In [2]:
data = df['raw-bytes'].to_numpy()

all_text = ' '.join(data)
all_words = all_text.split()

from collections import Counter
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
print('vocab length: ', len(vocab))

vocab_to_int = {word:i for i, word in enumerate(vocab[:10000], 1)}

pickle.dump(vocab_to_int, open('/content/drive/My Drive/Colab Notebooks/datasets/vocab_to_int.p', "wb"))

print('Vocab to Int save to drive')

limit_bytes_as_vocab = []
for raw in data:
    limit_bytes_as_vocab.append([word if word in vocab_to_int else 'unkown' for word in raw.split()])
print('Process Complete')

In [ ]:
for i, line in enumerate(limit_bytes_as_vocab):
  text_list = line
  if len(text_list)<1200:
    text_list.extend(['0']*(1200-len(text_list)))
  limit_bytes_as_vocab[i] = text_list

In [3]:
w2v_model = Word2Vec(min_count=1,
                     window=3,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     sg=1,
                     compute_loss=True,
                     workers=2)

t = time()
w2v_model.build_vocab(limit_bytes_as_vocab, progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [4]:
t = time()

w2v_model.train(limit_bytes_as_vocab, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.save('/content/drive/My Drive/Colab Notebooks/datasets/train-word2vec-model.model')
print('Model is Saved')

In [ ]:
# targets = df['target']
targets = targets.reshape(-1, 1)

test_index=[]
normal=np.random.choice(range(16117, 26763), 1000)
test_index.extend(set(normal))

Inflit=np.random.choice(range(6702, 16116), 900)
test_index.extend(set(Inflit))

http=np.random.choice(range(3594, 6701), 400)
test_index.extend(set(http))

bfssh=np.random.choice(range(0, 3593), 350)
test_index.extend(set(bfssh))

len(test_index)

train_x = []
test_x = []

for i in range(26764):
  if i in test_index:
    test_x.append(text_data[i])
  else:
    train_x.append(text_data[i])


train_y = targets[list(set(range(0, 26763)).symmetric_difference(set(test_index))), :]
test_y = targets[test_index, :]
# print('test_x: ', test_x.shape)
# print('train_x: ', train_x.shape)
print('Len of train data: ', len(train_x))
print('Len of test data: ', len(test_x))
print('train_y: ', train_y.shape)
print('test_y: ', test_y.shape)

Len of train data:  24244
Len of test data:  2520
train_y:  (24243, 1)
test_y:  (2520, 1)


In [ ]:
# test_x